In [3]:
# import pandas as pd
import pandas as pd

In [6]:
#read a file from this url "https://drive.google.com/file/d/1JNncMKuaXSwuQEp-S6wui4jiDcPGoCkU/view?usp=share_link" and save it as groups_data
groups_data = pd.read_csv("https://drive.google.com/file/d/1JNncMKuaXSwuQEp-S6wui4jiDcPGoCkU/view?usp=share_link")

# print the first 5 rows of the data
groups_data.head()


,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""A4rJUtaXt1-iRfP0q0Y3Kg"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>info_grupos_merged_separado_instituciones.csv - Google Drive</title><meta property=""og:title"" content=""info_grupos_merged_separado_instituciones.csv""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1JNncMKuaXSwuQEp-S6wui4jiDcPGoCkU/view?usp=share_link&amp;usp=embed_facebook""><meta property=""og:image"" content=""https://lh5.googleusercontent.com/aQtuaZBmB8lsvhuZO8xnjQrx_G0ErDvqqz8_KeNprEoU0sOY9-ujFqnUHaTWnsiC0v4=w1200-h630-p""><meta property=""og:image:width"" content=""1200""><meta property=""og:image:height"" content=""630""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans:300",400,500,"700"" nonce=""A4rJUtaXt1-iRfP0q0Y3Kg""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.03McfMPRSy4.L.X.O/am=AAg/d=0/rs=AO0039sJla-x6f-YFSw73-N7BaUKg9JCMQ"" nonce=""A4rJUtaXt1-iRfP0q0Y3Kg""><script nonce=""f6IOViZjukawy09bNrVX1g"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nfd""",docs-l2t:0,docs-shdn:0,"docs-tfh:""""",info_params:{},...,2.3,5.2,qr,126,0].7,[[null.2,null.198,null.199,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.hj89-rW3G9Y.O/rt=j/m=qabr,q_dnp,qapid/exm=qaaw,qadd,qaid,qein,qhaw,qhbr,qhch,qhga,qhid,qhin,qhpr/d=1/ed=1/rs=AA2YrTuy-g1QunQbD3MW84FOnc-xAfoKVw]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
0,try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,*/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
